In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220719082949.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949


In [3]:
!pip install tensorflow==1.14

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 46 kB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 488 kB 43.3 MB/s 
     |████████████████████████████████| 3.1 MB 34.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.14.0 which is incompatible.


In [4]:
import os
from os import path

import glob
import pandas as pd
import numpy as np

import shutil

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

import tensorflow as tf

# signal processing
from scipy import signal
from scipy.ndimage import label as sci_label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

# misc
import warnings

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [5]:
%matplotlib inline
%pylab inline
import warnings
import itertools    
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input, Dropout, BatchNormalization, GRU
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

sns.set_style('dark')

warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime', 'trapz']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
stimulus_desc_file = pd.read_excel('/content/drive/MyDrive/Emotion-RST/YAAD/data/ECG_GSR_Emotions/Stimulus_Description.xlsx')
stimulus_desc_file.to_csv('Stimulus_Description.csv', index = None, header=True)
stimulus_desc = pd.read_csv('Stimulus_Description.csv')

self_annotation_multimodal_file = pd.read_excel('/content/drive/MyDrive/Emotion-RST/YAAD/data/ECG_GSR_Emotions/Self-Annotation Labels/Self-annotation Multimodal_Use.xlsx')
self_annotation_multimodal_file.to_csv('Self-annotation Multimodal_Use.csv', index = None, header=True)
self_annotation_multimodal = pd.read_csv('Self-annotation Multimodal_Use.csv')
self_annotation_multimodal['annotation'] = 'M'
self_annotation_multimodal.rename(columns = {'V_Label':'Valence', 'A_Label':'Arousal', 'Four_Labels':'Four_Label'}, inplace = True)

self_annotation_singlemodal_file = pd.read_excel('/content/drive/MyDrive/Emotion-RST/YAAD/data/ECG_GSR_Emotions/Self-Annotation Labels/Self-annotation Single Modal_Use.xlsx')
self_annotation_singlemodal_file.to_csv('Self-annotation Single Modal_Use.csv', index = None, header=True)
self_annotation_singlemodal = pd.read_csv('Self-annotation Single Modal_Use.csv')
self_annotation_singlemodal['annotation'] = 'S'
self_annotation_singlemodal.rename(columns = {'Male':'Gender', 'Session Id':'Session ID', 'Video Id':'Video ID'}, inplace = True)

In [7]:
self_annotation_frames = [self_annotation_multimodal, self_annotation_singlemodal]
merged_dataframe = pd.concat(self_annotation_frames)
#merged_dataframe.head()
#merged_dataframe.tail()
table_frame = merged_dataframe.copy()

cols = []
cols.append('Raw Data')
for col in merged_dataframe.columns:
    cols.append(col)

In [8]:
arr_shape = 1000
raw_data_arr = []

def form_data(data_array = [], annotation = '', data_path = ''):
    #######
    #
    #     If anyone could help me improve this code please do so.
    #     Goals was to product the same data format of raw_dataframe DF below.
    #
    #######
    for filename in os.listdir(data_path):
            data = np.loadtxt(os.path.join(data_path, filename), delimiter=',')
            data = data[0:arr_shape]
            filenames = filename.split('ECGdata_')[1]
            filenames = filenames.split('.dat')[0]
            filenames = filenames.lower()
            s = filenames.split('s')[1].split('p');
            p = s[1].split('v')
            s = s[0]
            v = p[1]
            p = p[0]
            data_row = merged_dataframe.loc[(merged_dataframe['Session ID'] == int(s)) & 
                             (merged_dataframe['Participant Id'] == int(p)) & 
                             (merged_dataframe['Video ID'] == int(v)) &
                             (merged_dataframe['annotation'] == 'M')]
            stim_row = stimulus_desc.loc[(stimulus_desc['Session ID'] == int(s)) & 
                             (stimulus_desc['Video ID'] == int(v))]
            for index, row in data_row.iterrows():
              data_array.append([data, 
                                   row['Participant Id'], row['Session ID'], row['Video ID'],
                                   row['Name'], row['Age'], row['Gender'], row['Valence level'],
                                   row['Arousal level'], row['Dominance level'], row['Happy'],
                                   row['Sad'], row['Fear'], row['Anger'], row['Neutral'],
                                   row['Disgust'], row['Surprised'], row['Familiarity Score'],
                                   row['Emotion'], row['Valence'], row['Arousal'], row['Four_Label'],
                                   row['annotation'],  stim_row['Target Emotion'].iat[0]
                                   ])
    return data_array

    
raw_data_arr =  form_data(data_array = raw_data_arr, annotation = 'M', data_path = "/content/drive/MyDrive/Emotion-RST/YAAD/data/ECG_GSR_Emotions/Raw Data/Multimodal/ECG/")
raw_data_arr =  form_data(data_array = raw_data_arr, annotation = 'S', data_path = "/content/drive/MyDrive/Emotion-RST/YAAD/data/ECG_GSR_Emotions/Raw Data/Single Modal/ECG/")
cols.append('Target Emotion')
raw_dataframe = pd.DataFrame(raw_data_arr, columns = cols)
raw_dataframe.rename(columns = {'Participant Id':'Participant ID', 'annotation':'Modal', 'Four_Label':'Four label'}, inplace = True)
raw_dataframe['Familiarity Score'] = raw_dataframe['Familiarity Score'].fillna('Never watched')
raw_dataframe = raw_dataframe.replace(np.nan, '', regex=True)

In [9]:
plot_frame = raw_dataframe.copy()

plot_frame = plot_frame.drop(['Participant ID', 'Session ID', 'Familiarity Score', 'Age', 'Gender', 'Name'], axis = 1)
sad_data = plot_frame.loc[(plot_frame['Emotion'] == 'Sad') & (plot_frame['Target Emotion'] == 'sad')] 
fear_data = plot_frame.loc[(plot_frame['Emotion'] == 'Fear')  & (plot_frame['Target Emotion'] == 'fear')]
happy_data = plot_frame.loc[(plot_frame['Emotion'] == 'Happy') & (plot_frame['Target Emotion'] == 'happy')]
anger_data = plot_frame.loc[(plot_frame['Emotion'] == 'Anger') & (plot_frame['Target Emotion'] == 'anger')]
neutral_data = plot_frame.loc[(plot_frame['Emotion'] == 'Neutral') & (plot_frame['Target Emotion'] == 'neutral')]
mixed_data = plot_frame.loc[(plot_frame['Emotion'] == 'Mixed') & (plot_frame['Target Emotion'] == 'neutral')]
disgust_data = plot_frame.loc[(plot_frame['Emotion'] == 'Disgust') & (plot_frame['Target Emotion'] == 'disgust')]
surprised_data = plot_frame.loc[(plot_frame['Emotion'] == 'Surprise') & (plot_frame['Target Emotion'] == 'surprise')]

In [10]:
train_d_frame = plot_frame.copy().drop(['Video ID', 'Happy', 'Sad', 'Fear',
                                        'Anger', 'Neutral', 'Disgust', 'Surprised', 'Four label', 'Modal'
                                       ], axis = 1)
train_d_frame.rename(columns = {'Raw Data':'feature', 'Emotion':'emotion'}, inplace = True)
train_d_frame

,feature,Valence level,Arousal level,Dominance level,emotion,Valence,Arousal,Target Emotion
0,"[-4.96, -4.9633, -4.9583, -4.959, -4.9552, -4....",7,2,0,Happy,HV,LA,happy
1,"[-5.9868, -5.9908, -5.9927, -5.9869, -5.9778, ...",3,5,0,Sad,LV,LA,sad
2,"[2.732, 2.7337, 2.7454, 2.754, 2.7657, 2.7899,...",4,0,0,Mixed,LV,LA,fear
3,"[-18.619, -18.624, -18.615, -18.604, -18.616, ...",5,4,2,Mixed,LV,LA,sad
4,"[5.754, 5.7436, 5.8045, 5.8732, 5.806, 5.7849,...",3,2,1,Sad,LV,LA,anger
...,...,...,...,...,...,...,...,...
394,"[-1.8641, -1.8731, -1.8956, -1.8955, -1.8831, ...",5,4,4,Mixed,LV,LA,neutral
395,"[2.4423, 2.4517, 2.448, 2.4557, 2.4535, 2.4656...",4,4,9,Surprise,LV,LA,neutral
396,"[-20.306, -20.321, -20.322, -20.318, -20.309, ...",0,9,0,Fear,LV,HA,fear
397,"[14.621, 14.622, 14.612, 14.609, 14.612, 14.62...",1,2,1,Sad,LV,LA,anger


In [11]:
le = LabelEncoder()
train_d_frame['Valence'] = le.fit_transform(train_d_frame['Valence'])
train_d_frame['Arousal'] = le.fit_transform(train_d_frame['Arousal'])
train_d_frame.head()

,feature,Valence level,Arousal level,Dominance level,emotion,Valence,Arousal,Target Emotion
0,"[-4.96, -4.9633, -4.9583, -4.959, -4.9552, -4....",7,2,0,Happy,0,1,happy
1,"[-5.9868, -5.9908, -5.9927, -5.9869, -5.9778, ...",3,5,0,Sad,1,1,sad
2,"[2.732, 2.7337, 2.7454, 2.754, 2.7657, 2.7899,...",4,0,0,Mixed,1,1,fear
3,"[-18.619, -18.624, -18.615, -18.604, -18.616, ...",5,4,2,Mixed,1,1,sad
4,"[5.754, 5.7436, 5.8045, 5.8732, 5.806, 5.7849,...",3,2,1,Sad,1,1,anger


In [13]:
#1000个feature
x = np.array(train_d_frame['feature'].tolist())
y = np.array(train_d_frame['Arousal'].tolist())

sc = StandardScaler()
x = sc.fit_transform(x)

labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.8, random_state = 0) # Train 79

x_unlabeled, x_test, y_unlabeled, y_test = train_test_split( x_test,  y_test, test_size = 0.25, random_state = 0) # Unlabeled 240 x_test 80
print("x_train",x_train.shape)

x_train = x_train.reshape(( x_train.shape[0], x_train.shape[1],1))
x_test= x_test.reshape(( x_test.shape[0], x_test.shape[1],1))
num_labels = y.shape[1]
print("x_train",x_train.shape)
print("x_unlabeled",x_unlabeled.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

x_train (79, 1000)
x_train (79, 1000, 1)
x_unlabeled (240, 1000)
x_test (80, 1000, 1)
y_test (80, 2)


In [16]:
i_lstm = Input(shape=(x_train.shape[1],1))

x_lstm = LSTM(32, return_sequences=True)(i_lstm)
x_lstm = Flatten()(x_lstm)
y_lstm = Dense(2, activation='softmax')(x_lstm)

model_lstm = Model(i_lstm, y_lstm)

model_lstm.summary()
adam = Adam(learning_rate=0.001)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
mc = ModelCheckpoint('./best_rstlstm_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
lr_schedule = LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
                                    
model_lstm.compile(optimizer=adam,
                  loss=['categorical_crossentropy'],
                  metrics=['accuracy'])
        
lstm_h = model_lstm.fit(x_train, y_train,
                   batch_size=8,
                   validation_data=(x_test, y_test),
                   epochs=20,
                   callbacks=[es, mc, lr_schedule])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 1)]         0         
_________________________________________________________________
lstm (LSTM)                  (None, 1000, 32)          4352      
_________________________________________________________________
flatten (Flatten)            (None, 32000)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 64002     
Total params: 68,354
Trainable params: 68,354
Non-trainable params: 0
_________________________________________________________________
Train on 79 samples, validate on 80 samples


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
72/79 [==========================>...] - ETA: 0s - loss: 1.2412 - acc: 0.4028
Epoch 00001: val_acc improved from -inf to 0.48750, saving model to ./best_rstlstm_model.h5
79/79 [==============================] - 12s 146ms/sample - loss: 1.1922 - acc: 0.3924 - val_loss: 0.7107 - val_acc: 0.4875
Epoch 2/20
72/79 [==========================>...] - ETA: 0s - loss: 0.7648 - acc: 0.5972
Epoch 00002: val_acc improved from 0.48750 to 0.67500, saving model to ./best_rstlstm_model.h5
79/79 [==============================] - 7s 89ms/sample - loss: 0.7807 - acc: 0.6076 - val_loss: 0.9000 - val_acc: 0.6750
Epoch 3/20
72/79 [==========================>...] - ETA: 0s - loss: 0.7333 - acc: 0.7361
Epoch 00003: val_acc did not improve from 0.67500
79/79 [==============================] - 7s 89ms/sample - loss: 0.7719 - acc: 0.7215 - val_loss: 0.7754 - val_acc: 0.5750
Epoch 4/20
72/79 [==========================>...] - ETA: 0s - loss: 0.6467 - acc: 0.6944
Epoch 00004: val_acc did not improve fr

In [17]:
test_accuracy = model_lstm.evaluate(x_test, y_test, verbose = 0)
print(test_accuracy)

[0.6542727470397949, 0.6625]


In [ ]:
#x_train, x_test, y_train, y_test

#x_unlabeled, x_test, y_unlabeled, y_test

# RSL 

In [18]:
#1000个feature
dataset = np.array(train_d_frame['feature'].tolist())


new_dataset = []
for i in range(len(train_d_frame)):
  features = train_d_frame['feature'].iloc[i].tolist()
  label = train_d_frame['Arousal'][i]
  new_row = features.copy()
  new_row.append(label)
  #print(len(new_row),new_row)
  new_dataset.append(new_row)

new_dataset = np.array(new_dataset)

new_dataset.shape

(399, 1001)

In [19]:
x = new_dataset[:,:-1]
y = new_dataset[:,-1:]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.8, random_state = 100) # Train 79

x_unlabeled, x_test, y_unlabeled, y_test = train_test_split( x_test,  y_test, test_size = 0.25, random_state = 100) # Unlabeled 240 x_test 80

In [21]:
labeled_dataset_ = np.column_stack((x_train, y_train))
label_ =labeled_dataset_[:,-1:].reshape(-1).tolist()
label_.count(1)
label_.count(0)

26

In [22]:
# Data manipulation
import pandas as pd

# Visualization
import plotly.express as px

# Sklearn
from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.svm import SVC # for Support Vector Classification baseline model
from sklearn.semi_supervised import SelfTrainingClassifier # for Semi-Supervised learning
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.metrics import f1_score

from sklearn.utils import shuffle
import math
import os
import numpy as np

import random

import time
import datetime



In [23]:
## ENVIRONMENT Hyperparameters
state_size = 1002
#action_size = env.action_space.n
action_size = 2

## TRAINING Hyperparameters
max_episodes = 300
learning_rate = 0.01
gamma = 0.95 # Discount rate
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / (std) if std!=0 else (discounted_episode_rewards - mean)
    
    return discounted_episode_rewards

with tf.name_scope("inputs"):
    input_ = tf.placeholder(tf.float32, [None, state_size], name="Input_")
    actions = tf.placeholder(tf.int32, [None, action_size], name="Actions")
    discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,], name="Discounted_episode_rewards")
    
    # Add this placeholder for having this variable in tensorboard
    mean_reward_ = tf.placeholder(tf.float32 , name="Mean_reward")

    with tf.name_scope("fc1"):
        fc1 = tf.contrib.layers.fully_connected(inputs = input_,
                                                num_outputs = 10,
                                                activation_fn=tf.nn.relu,
                                                weights_initializer=tf.contrib.layers.xavier_initializer())

    with tf.name_scope("fc2"):
        fc2 = tf.contrib.layers.fully_connected(inputs = fc1,
                                                num_outputs = action_size,
                                                activation_fn= tf.nn.relu,
                                                weights_initializer=tf.contrib.layers.xavier_initializer())
    
    with tf.name_scope("fc3"):
        fc3 = tf.contrib.layers.fully_connected(inputs = fc2,
                                                num_outputs = action_size,
                                                activation_fn= None,
                                                weights_initializer=tf.contrib.layers.xavier_initializer())

    with tf.name_scope("softmax"):
        action_distribution = tf.nn.softmax(fc3)

    with tf.name_scope("loss"):
        # tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the result after applying the softmax function
        # If you have single-class labels, where an object can only belong to one class, you might now consider using 
        # tf.nn.sparse_softmax_cross_entropy_with_logits so that you don't have to convert your labels to a dense one-hot array. 
        neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = actions)
        loss = tf.reduce_mean(neg_log_prob * discounted_episode_rewards_) 
        
    
    with tf.name_scope("train"):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Setup TensorBoard Writer
writer = tf.summary.FileWriter("/tensorboard/pg/1")

## Losses
tf.summary.scalar("Loss", loss)

## Reward mean
tf.summary.scalar("Reward_mean", mean_reward_)

write_op = tf.summary.merge_all()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [24]:
import os 
os.environ["SDL_VIDEODRIVER"] = "dummy"

from sklearn.metrics import accuracy_score,precision_score, \
recall_score,f1_score,cohen_kappa_score

In [25]:
#归一化
def normalization(data):
  _range = np.max(data) - np.min(data)
  return (data - np.min(data)) / _range

# 标准化
def standardization(data):
  mu = np.mean(data, axis=0)
  sigma = np.std(data, axis=0)
  return (data - mu) / sigma
def get_svc_model():
  svc_model = SVC(kernel='rbf', 
    probability=True, 
    C=1.0, # default = 1.0
    gamma='scale', # default = 'scale'
    random_state=1)
  return svc_model
def predict_by_SVM(clf,unlabeled_data_x):
  class_probas = clf.predict_proba(unlabeled_data_x)
  predicted_labels = clf.predict(unlabeled_data_x)
  return class_probas, predicted_labels

def concat_probas_with_instances(unlabeled_bag,class_probas):
  states = np.concatenate((unlabeled_bag,class_probas),axis=1)
  return states

def to_batch_states(states):
  batch_states=[]
  dt = np.dtype('float32')
  for i in range(len(states)):
    batch_states.append(np.array(states[i],dtype = dt))
  batch_states = standardization(batch_states)
  return batch_states


##1002 base on (features1000+num_class2)
def decide_actions(sess,states):
	batch_actions = []
	myactions = []
	for state in states:
		#print(state.reshape([1,1002]))
		action_probability_distribution = sess.run(action_distribution, feed_dict={input_: state.reshape([1,1002])})
		#print("action_probability_distribution",action_probability_distribution)
		action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob
		action_ = np.zeros(action_size)
		action_[action] = 1
		batch_actions.append(action_)
		myactions.append(action)
	#array=numpy.array(list)
	return batch_actions,myactions

def get_selected_instances(unlabeled_bag,batch_actions,predicted_labels):
  #unlabeled_bag = unlabeled_bag.tolist()
  bag_selected = []
  bag_selected_labels = []
  for i in range(len(unlabeled_bag)):
    if(batch_actions[i] == 1):
      bag_selected.append(unlabeled_bag[i])
      bag_selected_labels.append(predicted_labels[i])
  return bag_selected,bag_selected_labels

def cal_reward(clf,last_clf,test_dataset):
  x = test_dataset[:,:2]
  y = test_dataset[:,-1:]
  
  score1 = clf.score(x,y)
  score2 = last_clf.score(x,y)
                     
  y_pred1 = clf.predict(x)
  f1_1 = f1_score(y, y_pred1, average='macro')
  
  y_pred2 = last_clf.predict(x)
  f1_2 = f1_score(y, y_pred2, average='macro')

  reward1 = score1 - score2
  reward2 = f1_1 - f1_2 #TO consider (score1-score2)*(f1_1 - f1_2)
  reward = reward1+reward2

  #print(cohen_kappa_score(y, y_pred1),cohen_kappa_score(y, y_pred2))
  #print(score1,score2)
  #print(f1_1,f1_2)
  return reward
def cal_performance(clf,last_clf,test_dataset_x,test_dataset_y):
  x = test_dataset_x
  y = test_dataset_y
  score1 = clf.score(x,y)
  score2 = last_clf.score(x,y)
                     
  y_pred1 = clf.predict(x)
  f1_1 = f1_score(y, y_pred1, average='macro')
  
  y_pred2 = last_clf.predict(x)
  f1_2 = f1_score(y, y_pred2, average='macro')

  return score1,score2,f1_1,f1_2

def pop_from_unlabeled_data(unlabeled_data,bag_selected):
  """a1=np.asarray(unlabeled_data)
  a2=np.asarray(bag_selected)

  a1_rows = a1.view([('', a1.dtype)] * a1.shape[1])
  a2_rows = a2.view([('', a2.dtype)] * a2.shape[1])#IndexError: tuple index out of range
  unlabeled_data = np.setdiff1d(a1_rows, a2_rows).view(a1.dtype).reshape(-1, a1.shape[1])"""

  a = unlabeled_data
  b = bag_selected
  ret = []
  for i in a:
    if i not in b:
         ret.append(i)
  unlabeled_data = ret
  return unlabeled_data
def split_unlabeled_bags(U,max_batches):
  every_epoch_num = math.floor((len(U)/max_batches))
  unlabeled_bags = []
  for k in range(max_batches):
      if k < (max_batches-1):
          Bag_k = U[every_epoch_num * k: every_epoch_num * (k + 1)]
          unlabeled_bags.append(Bag_k)
      else:
          Bag_k = U[every_epoch_num * k:]
          unlabeled_bags.append(Bag_k)

  return unlabeled_bags



In [26]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import svm
# 标准化
def standardization(data):
  mu = np.mean(data, axis=0)
  sigma = np.std(data, axis=0)
  return (data - mu) / sigma

def pca_transform(X,PCA_N):
  X_ = standardization(np.array(X.tolist()))
  pca = PCA(n_components=PCA_N)
  pca.fit(X_)
  X_1 = pca.fit_transform(X_)
  #print(X_1.shape)
  return X_1
    
def train_Classifier(X_train ,y_train ,do_pca):
  if(do_pca==True):
    X_train = pca_transform(X_train,PCA_N=10)
  #print(X_train.shape,X_train)
  #print(y_train.shape,y_train)
  random_state=1
  # 训练模型
  model = OneVsRestClassifier(svm.SVC(kernel='linear',probability=True,random_state=random_state))
  clt = model.fit(X_train,y_train)
  
  return clt

def evaluate_Classifier(clt,X_test,y_test,do_pca):
  if(do_pca==True):
    X_test = pca_transform(X_test,PCA_N=10)
  proba = clt.predict_proba(X_test)
  #print(proba)
  y_test_pred = clt.predict(X_test)
  ov_acc = metrics.accuracy_score(y_test_pred,y_test)
  #print("overall accuracy: %f"%(ov_acc))
  #print("===========================================")
  acc_for_each_class = metrics.precision_score(y_test,y_test_pred,average=None)
  #print("acc_for_each_class:\n",acc_for_each_class)
  #print("===========================================")
  avg_acc = np.mean(acc_for_each_class)
  #print("average accuracy:%f"%(avg_acc))
  #print(y_test_pred)
  #print(y_test)
  return proba, y_test_pred, ov_acc, acc_for_each_class, avg_acc

In [27]:
def decide_labels(class_probas):
  #print("Decide_labels:")
  class_dis = np.array([class_probas[0]])
  l = np.random.choice(range(class_dis.shape[1]), p=class_dis.ravel())
  #print(l)
  p_labels=[]
  for proba in class_probas:
    class_dis = np.array([proba])
    label = np.random.choice(range(class_dis.shape[1]), p=class_dis.ravel())
    if(label==0):
      label=0
    elif(label==1):
      label=1

    #print(class_dis,label)
    p_labels.append(label)
  return p_labels


In [28]:
#x_train, x_test, y_train, y_test

#x_unlabeled, x_test, y_unlabeled, y_test
#合并feature和label

labeled_dataset = np.column_stack((x_train, y_train))
unlabeled_dataset = np.column_stack((x_unlabeled, y_unlabeled))
test_dataset = np.column_stack((x_test, y_test))

"""
labeled_dataset = x_train
unlabeled_dataset = x_unlabeled
test_dataset = x_test"""
unlabeled_dataset.shape

(240, 1001)

In [31]:
DO_PCA=False
base_clt = train_Classifier(X_train=labeled_dataset[:,:-1] ,y_train=labeled_dataset[:,-1:].reshape(-1),do_pca=DO_PCA)

base_proba,base_y_test_pred,base_ov_acc,base_acc_for_each_class,base_avg_acc = evaluate_Classifier(base_clt,X_test=test_dataset[:,:-1],y_test=test_dataset[:,-1:].reshape(-1),do_pca=DO_PCA)

In [32]:
base_ov_acc

0.6375

In [33]:
base_acc_for_each_class

array([0.4       , 0.69230769])

In [34]:
base_avg_acc

0.5461538461538462

In [ ]:

#print(labeled_dataset.shape)
max_iterations = 100
max_episodes = 3
max_batches = 7
DO_PCA=False
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for iteration in range(max_iterations):
        
        episode_rewards_sum = 0

        #observation = env.reset()#重置整个环境
        unlabeled_data = unlabeled_dataset.tolist()#初始化无标签样本池
        pseudo_labeled_data = []#初始化假定标签样本池

        print("第",iteration+1,"次迭代",end = "")
        svc_model = get_svc_model()
        
        base_clt = train_Classifier(X_train=labeled_dataset[:,:-1] ,y_train=labeled_dataset[:,-1:].reshape(-1),do_pca=DO_PCA)
        base_proba,base_y_test_pred,base_ov_acc,base_acc_for_each_class,base_avg_acc = evaluate_Classifier(base_clt,X_test=test_dataset[:,:-1],y_test=test_dataset[:,-1:].reshape(-1),do_pca=DO_PCA)

        for episode in range(max_episodes):
          print("第",episode+1,"episode")
          train_data_ = np.array(labeled_dataset.tolist() + pseudo_labeled_data)
          episode_clt = train_Classifier(X_train=train_data_[:,:-1] ,y_train=train_data_[:,-1:].reshape(-1),do_pca=DO_PCA)
          episode_proba,episode_y_test_pred,episode_ov_acc,episode_acc_for_each_class,episode_avg_acc = evaluate_Classifier(episode_clt,X_test=test_dataset[:,:-1],y_test=test_dataset[:,-1:].reshape(-1),do_pca=DO_PCA)
          
          U = np.array(unlabeled_data)[:,:-1]
          random.shuffle(U)
          unlabeled_bags = split_unlabeled_bags(U,max_batches)

          for batch in range(max_batches):
            print("   batch",batch+1,"------")
            if(batch==0):
              #print("last_clf = episode_clf")
              last_clt = episode_clt
            unlabeled_bag = unlabeled_bags[batch]
            unlabeled_bag = np.array(unlabeled_bag)
            #print(unlabeled_bag.shape)#30
            class_probas = episode_clt.predict_proba(unlabeled_bag)
            predicted_labels = episode_clt.predict(unlabeled_bag)#y_unlabeled_pred
            p_labels = decide_labels(class_probas)
            predicted_labels = p_labels
            #基于class_probas，随机决定labels
            #print(predicted_labels)
            #print(p_labels)
            #input()

            states = concat_probas_with_instances(unlabeled_bag,class_probas)
            #print(states.shape)#32
            #print(states)
            #print(predicted_labels)

            #

            
            batch_states = to_batch_states(states)
            batch_actions,myactions = decide_actions(sess,batch_states)
            #print(myactions)
            bag_selected , bag_selected_labels = get_selected_instances(unlabeled_bag,myactions,predicted_labels)
            #print(batch_states)
            
            unlabeled_data = pop_from_unlabeled_data(unlabeled_data,bag_selected)
            bag_selected_labeled = np.concatenate((bag_selected , np.array(bag_selected_labels).reshape(len(bag_selected_labels),1)),axis=1)
            #bag_selected_labeled = np.concatenate((bag_selected , np.array(bag_selected_labels).reshape(len(bag_selected_labels),1)),axis=1)
            print("bag_selected_labeled",len(bag_selected_labeled),bag_selected_labeled)

            pseudo_labeled_data = pseudo_labeled_data + bag_selected_labeled.tolist()
            print("add",len(bag_selected_labeled),"instance to peseudo set"," now peseudo has",len(pseudo_labeled_data)," instances") 
            train_data_temp = np.array(pseudo_labeled_data + labeled_dataset.tolist())
          
            temp_clt = train_Classifier(X_train=train_data_temp[:,:-1] ,y_train=train_data_temp[:,-1:].reshape(-1),do_pca=DO_PCA)
            temp_proba,temp_y_test_pred,temp_ov_acc,temp_acc_for_each_class,temp_avg_acc = evaluate_Classifier(temp_clt,X_test=test_dataset[:,:-1],y_test=test_dataset[:,-1:].reshape(-1),do_pca=DO_PCA)
            last_proba,last_y_test_pred,last_ov_acc,last_acc_for_each_class,last_avg_acc = evaluate_Classifier(last_clt,X_test=test_dataset[:,:-1],y_test=test_dataset[:,-1:].reshape(-1),do_pca=DO_PCA)
            
            last_clt = temp_clt

            print("temp",temp_ov_acc,temp_acc_for_each_class,temp_avg_acc)
            print("last",last_ov_acc,last_acc_for_each_class,last_avg_acc)

            reward = (temp_acc_for_each_class[0]-last_acc_for_each_class[0])+ (temp_acc_for_each_class[1]-last_acc_for_each_class[1])
            #reward = temp_ov_acc - last_ov_acc
            batch_rewards = np.full(len(states),reward)
            discounted_batch_rewards = batch_rewards
            #print("batch_states ",np.vstack(np.array(batch_states)).shape,type(np.vstack(np.array(batch_states))) , np.vstack(np.array(batch_states)))
            #print("batch_actions ",np.vstack(np.array(batch_actions)).shape,type(np.vstack(np.array(batch_actions))) , np.vstack(np.array(batch_actions))) 
            #print("discounted_batch_rewards",discounted_batch_rewards.shape,type(discounted_batch_rewards),discounted_batch_rewards)
            loss_, _ = sess.run([loss, train_opt], feed_dict={input_: np.vstack(np.array(batch_states)),
                                    actions: np.vstack(np.array(batch_actions)),																						
                                    discounted_episode_rewards_: discounted_batch_rewards
                                          })
            mean_reward = np.float64(discounted_batch_rewards[0]-0.001)
            """
            summary = sess.run(write_op, feed_dict={input_: np.vstack(np.array(batch_states)),
                                  actions: np.vstack(np.array(batch_actions)),
                                  discounted_episode_rewards_: discounted_batch_rewards,
                                  mean_reward_: mean_reward
                                })
            writer.add_summary(summary, batch)
            writer.flush()"""



第 1 次迭代第 1 episode
   batch 1 ------
bag_selected_labeled 20 [[-18.705   -18.669   -18.642   ... -18.719   -18.708     1.     ]
 [  8.7897    8.7249    8.6862  ...   8.4915    8.503     0.     ]
 [  8.7897    8.7249    8.6862  ...   8.4915    8.503     1.     ]
 ...
 [-18.561   -18.561   -18.557   ... -18.589   -18.59      1.     ]
 [ -0.65165  -0.64549  -0.65426 ...  -0.65385  -0.65515   1.     ]
 [-19.613   -19.619   -19.608   ... -19.631   -19.627     0.     ]]
add 20 instance to peseudo set  now peseudo has 20  instances
temp 0.6375 [0.38461538 0.68656716] 0.5355912743972445
last 0.6375 [0.4        0.69230769] 0.5461538461538462
   batch 2 ------
bag_selected_labeled 22 [[  0.4997    0.4956    0.5031  ...   0.48345   0.4786    1.     ]
 [-19.925   -19.934   -19.924   ... -19.875   -19.881     1.     ]
 [ 16.51     16.496    16.5     ...  16.418    16.433     1.     ]
 ...
 [ -0.22412  -0.22732  -0.22016 ...  -0.20387  -0.21314   1.     ]
 [-18.702   -18.694   -18.692   ... -18.827 

In [ ]:
temp_acc_for_each_class = [0.4       , 0.76363636]
last_acc_for_each_class = [0.34782609 ,0.73684211]
reward = (temp_acc_for_each_class[0]-last_acc_for_each_class[0])*(temp_acc_for_each_class[1]-last_acc_for_each_class[1])
reward

0.0013979607880175023